Imports of libraries and necessary functions

In [1]:
import io
import pandas as pd
import tensorflow as tf
import numpy as np

from google.colab import files
from numpy import mean
from numpy import std
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from numpy import asarray
from keras import backend as K

Source file (output of the SQL script) download

In [2]:
!wget https://anonymous.4open.science/r/mac_gyver_predictor-D307/initial_dataset_from_sql.csv

--2022-05-27 00:46:07--  https://anonymous.4open.science/r/mac_gyver_predictor-D307/initial_dataset_from_sql.csv
Resolving anonymous.4open.science (anonymous.4open.science)... 172.67.183.76, 104.21.18.195, 2606:4700:3035::ac43:b74c, ...
Connecting to anonymous.4open.science (anonymous.4open.science)|172.67.183.76|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /api/repo/mac_gyver_predictor-D307/file/initial_dataset_from_sql.csv [following]
--2022-05-27 00:46:07--  https://anonymous.4open.science/api/repo/mac_gyver_predictor-D307/file/initial_dataset_from_sql.csv
Reusing existing connection to anonymous.4open.science:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘initial_dataset_from_sql.csv.1’

initial_dataset_fro     [ <=>                ]  33.09K  --.-KB/s    in 0s      

2022-05-27 00:46:07 (110 MB/s) - ‘initial_dataset_from_sql.csv.1’ saved [33888]



Initial load and processing of the source file

In [3]:
df = pd.read_csv('/content/initial_dataset_from_sql.csv', delimiter=';', header=None)
df.columns=['Constituent', 'Emergent Behavior']

compose_dataset = pd.DataFrame()

uniqueValues = df['Emergent Behavior'].unique()

for i in range (len(uniqueValues)):
  constituents = df[df['Emergent Behavior'] == uniqueValues[i]][['Constituent']]
  constituents_uniqueValues = pd.DataFrame(constituents['Constituent'].unique())
  constituents_Transposed = constituents_uniqueValues.T
  constituents_Transposed.insert(0, 'Emergent Behavior', uniqueValues[i])
  compose_dataset = pd.concat([compose_dataset, constituents_Transposed], ignore_index=False, axis=0)

#constituents_Transposed = constituents_Transposed.reset_index()
#del constituents_Transposed['index']

compose_dataset = compose_dataset.reset_index(drop=True)
final_dataset = compose_dataset.replace(np.nan, '')

Initial dataframe processing for visualization optimization. In the resulting dataframe, the indexes are the SoS represented by their constituents. Columns labes are the possible emergent behaviors, where a value "zero" indicates the absence of that emergent behavior in that SoS and the value "one" represents the occurrence of that emergent behavior in that SoS.

In [4]:
constituents_aggregate_list = []
constituents_iterate_list = []

for i in range (len(final_dataset)):
  for j in range (0, len(final_dataset.columns) - 1):
    if (final_dataset.iloc[i][j] != ''):
      constituents_iterate_list.append(final_dataset.iloc[i][j])
  constituents_aggregate_list.append(constituents_iterate_list)
  constituents_iterate_list = []

df_constituents_aggregate_list = pd.DataFrame([constituents_aggregate_list])
df_constituents_aggregate_list_transposed = df_constituents_aggregate_list.T

emergent_behaviors_column = pd.DataFrame(final_dataset['Emergent Behavior'])

base_dataset = pd.concat([df_constituents_aggregate_list_transposed, emergent_behaviors_column], axis=1)
emergent_behaviors_column_transposed = emergent_behaviors_column.T

unique_sets = [list(x) for x in set(tuple(x) for x in constituents_aggregate_list)]
df_unique_sets = pd.DataFrame([unique_sets])
df_unique_sets_transposed = df_unique_sets.T
df_unique_sets_transposed.rename(columns={0: 'Constituents'}, inplace=True)

deep_learning_base_dataset = pd.DataFrame(np.zeros((len(df_unique_sets_transposed), len(emergent_behaviors_column_transposed.columns))))
working_deep_learning_base_dataset = deep_learning_base_dataset

deep_learning_base_dataset.columns = base_dataset['Emergent Behavior']

deep_learning_base_dataset = deep_learning_base_dataset.set_index(df_unique_sets_transposed['Constituents'])

for i in range (len(deep_learning_base_dataset)):
  filter = base_dataset[base_dataset[0].isin([deep_learning_base_dataset.index[i]])]
  filter_array = filter['Emergent Behavior'].to_numpy()
  #print(filter_array)
  for j in range (len(deep_learning_base_dataset.columns)):
    for k in range (len(filter_array)):
      if (filter_array[k] == deep_learning_base_dataset.columns[j]):
        #print('vou mudar valor')
        working_deep_learning_base_dataset.iloc[i][j] = 1
      else:
        continue

In [5]:
#working_deep_learning_base_dataset.insert(0, 'Constituents', df_unique_sets_transposed['Constituents'])
working_deep_learning_base_dataset = working_deep_learning_base_dataset.set_index(df_unique_sets_transposed['Constituents'])

working_deep_learning_base_dataset.to_csv('base_dataset_for_Deep_Learning_algorithm.csv')

In [6]:
working_deep_learning_base_dataset

Emergent Behavior,load-generation balance,control the battery and enhance the operation of the system,decrease the power consumption from power utility by using available energy from the solar panels,supply power power to the electric load,supply power to homes even in the event of an electric grid outage,to ensure the comfort of its occupants by enhancing the automation to relief them from manual control of different services while increasing the energy savings,minimize the average vehicle waiting time in the network,minimize the local waiting time for vehicles,"north-to-south, red-to-green transition times",balance between energy production and consumption,...,Different responses according to different demand/response schemes (price-based),decision about the best control design of the Smart Grid,optimizations of the decisions of the Smart Grid given the best control design,to proactively plan the DC operation as a system to be able to exploit its latent flexible energy to achieve a deeper and more efficient integration with the local Smart Grid.,"to use as much as possible of the locally produced renewable energy and transact energy to decrease operational costs, optimizing its integration in smart grid advanced context.",system disruption responses and minimization of the overall costs of satisfying total demand from the system in these events,agents perform different functions based on their unique advantages relative to other agents postdisruption,"coordination mechanism, dynamic reorganization and coordination of the network under a disruption",Negotiation and collaboration for resource allocation and demand efficiency,Resolution of potencial conflicts
Constituents,,,,,,,,,,,,,,,,,,,,,
"[Group (Representative) Agent - collectives, (Prosumers) Households with Photovoltaic cells and wind turbines]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"[Photovoltaic System, electric vehicles]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"[Heating/Ventilation and Air-Conditioning system (HVAC), Building Management System (BMS), Lighting CS, Smart Grid CS, Audiovisual Management CS, Online Scheduling CS, Fire Detection CS, Security Surveillance CS, Emergency Management CS]",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"[Electric Appliances (lights/refrigerators/freezers/washers/dyers/water heaters/baseload), Heating/Ventilation and Air-Conditioning system (HVAC), one heating system, smart home applications utilizing IoT technologies, Central controller]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"[one heating system, one home automation system, one vehicle parking system]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"[Diesel Generator Agent, Transfer Switch, Storage Agent, Battery storages, IT Servers, Thermal Storage Device (TES), Cooling System, Power Distribution Unit (PDU), Wind Energy system]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
"[Electric Load, Photovoltaic System, Wind Energy system]",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"[Commercial Greenhouse Growers (CGGs), Virtual Power Plant (VPP) - Generator units/Controllable loads/Storage systems]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
"[Market Agent, Storage Agent, Optimizer Agent, Group (Representative) Agent - collectives, (Prosumers) Households with Photovoltaic cells and wind turbines, Service providers]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
